### 1. Unarchive scans

In [23]:
%%bash
for subj in sub-co11; do
echo "$subj"
    for sess in pre; do
        tar -xzf ./data/DONE/$subj/$sess/raw/* -C ./data/DONE/$subj/$sess/raw;
        echo "$sess"
    done
done

sub-co11
pre


### 2. Create a temp directory for relevant DICOMs

In [24]:
%%bash
for subj in sub-co11; do
echo "$subj"
    for sess in pre; do
        cd /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/
        mr=mr
        scans=("$(ls -d $mr*)")
        mkdir temp
        for f in $scans; do
            mkdir temp/temp_$f
        done
    done
done

sub-co11


### 3. Convert DICOMs to NIFTIs

In [25]:
import glob
import dicom2nifti
subjects = ['sub-co11']#,'sub-bi06','sub-bi07','sub-bi08','sub-ar01','sub-ar05','sub-ar06','sub-ar08','sub-ar09','sub-ar12','sub-co01']
files = ['0003','0007','0008','0009','0010','0011','0012','0013']

for subj in subjects:
    print(subj)
    for i in files:
        for input in glob.glob('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subj)+'/pre/raw/*/*/mr_'+str(i)+'/'):
            print(input)
        for output in glob.glob('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subj)+'/pre/raw/*/*/temp/temp_mr_'+str(i)+'/'):
            print(output)
        dicom2nifti.convert_directory(input,output)

sub-co11
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/mr_0003/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/temp/temp_mr_0003/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/mr_0007/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/temp/temp_mr_0007/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/mr_0008/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/temp/temp_mr_0008/
/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/sub-co11/pre/raw/JONES_MADALYN_ELIZABETH-08198214/20211212-30282/mr_0009/
/S

### 4. Move and rename NIFTIs to working directories

In [26]:
%%bash
for subj in sub-co11; do
echo "$subj"
    for sess in pre; do
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0007/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-1_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0008/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-2_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0009/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-3_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0010/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-4_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0011/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-5_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0012/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_run-6_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0013/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/func/${subj}_task_visloc_bold.nii.gz
        mv /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/raw/*/*/temp/temp_mr_0003/* /System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/$subj/$sess/anat/${subj}_T1w.nii.gz
    done
done

sub-co11


### 5. Format scan onset timing files pulled from the trialmats outputted after each run

In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import glob
from scipy import stats
import numpy as np
import pandas as pd
import warnings
%pylab

subjs='sub-co11'
sessions ='pre'
run=6
script=383

#this will output the onsets for the instructions
trialmat = pd.read_csv('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subjs)+'/'+str(sessions)+'/trialmats/'+str(subjs)+'_sess_'+str(sessions)+'_run_'+str(run)+'_script_'+str(script)+'.csv')
condX = pd.DataFrame(trialmat['instruction.started']-6) #when running the glm, I ignore the first 4 TRs (6s), so here I want to modify the timings by 6seconds
condX = condX.iloc[:,0:1]
condX.insert(1, 2, 1.5) # 1.5 second length
condX.insert(2, 3, 1)
np.savetxt('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subjs)+'/'+str(sessions)+'/model/onset_timings/' + 'run' + str(run) + os.sep + 'cond_instruction' + '.tsv', condX.values, fmt='%1.3f',delimiter='\t')
                
#this will output the onsets for the individual conditions
trialmat = pd.read_csv('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subjs)+'/'+str(sessions)+'/trialmats/'+str(subjs)+'_sess_'+str(sessions)+'_run_'+str(run)+'_script_'+str(script)+'.csv')
for i in range(20):
    condX = pd.DataFrame(trialmat.loc[trialmat.uniquetrial == i,'mov.started']-6)
    condX = condX.iloc[:,0:1]
    condX.insert(1, 2, 2) # 2.0 second length
    condX.insert(2, 3, 1)
    np.savetxt('/System/Volumes/Data/misc/data19/schonehr/ProControl/data/DONE/'+str(subjs)+'/'+str(sessions)+'/model/onset_timings/' + 'run' + str(run) + os.sep + 'cond' + str(i) + '.tsv', condX.values, fmt='%1.3f',delimiter='\t')

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
